In [255]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from IPython.display import IFrame
from IPython.display import display

In [256]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
%matplotlib inline

In [257]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df.pkl')

In [258]:
xg_df = pd.read_pickle('/Users/andrewpeters/GitHub/understat/match_level_player_stats.pkl')

In [261]:
fpl_replace_dict = {}

In [262]:
fpl_replace_dict['Bojan_Krkic'] = 'Bojan_Krkic Perez'
fpl_replace_dict['Ahmed El-Sayed_Hegazy'] = 'Ahmed El-Sayed_Hegazi'

In [263]:
#look at players from understat who are not showing up in the fpl data. Don't want to look at symmetric difference, because understat only has data for people who played

unmatched_names = [player for player in xg_df.player.unique() if player not in df.player.unique()]

In [264]:
name_replace_dict = {}
no_results = []
multi_results = {}

mylist = df.player.unique()

for name in unmatched_names:
    r = re.compile(f'.*{name}*')
    results = list(filter(r.match, mylist))
    if len(results) == 1:
        name_replace_dict[name] = results[0]
        print(f'searched for {name}, and found {results[0]} as a replacement')
    elif len(results) > 1:
        multi_results[name] = results
    else:
        no_results.append(name)

searched for Richarlison, and found Richarlison_de Andrade as a replacement
searched for Nyom, and found Allan-Roméo_Nyom as a replacement
searched for Oriol_Romeu, and found Oriol_Romeu Vidal as a replacement
searched for David_Luiz, and found David_Luiz Moreira Marinho as a replacement
searched for Ederson, and found Ederson_Santana de Moraes as a replacement
searched for Danilo, and found Danilo Luiz_da Silva as a replacement
searched for Jesús_Gámez, and found Jesús_Gámez Duarte as a replacement
searched for Yaya_Touré, and found Gnegneri Yaya_Touré as a replacement
searched for Fousseni_Diabate, and found Fousseni_Diabaté as a replacement
searched for Kenedy, and found Robert Kenedy_Nunes do Nascimento as a replacement
searched for Kepa, and found Kepa_Arrizabalaga as a replacement
searched for Fabri, and found Fabricio_Agosto Ramírez as a replacement
searched for Junior_Hoilett, and found David Junior_Hoilett as a replacement
searched for Jonny, and found Jonny_Evans as a replace

In [265]:
mylist = df.player.unique()

still_no_results = []

for full_name in no_results:
    name = full_name.split('_')[-1]
    r = re.compile(f'.*{name}*')
    results = list(filter(r.match, mylist))
    if len(results) == 1:
        name_replace_dict[full_name] = results[0]
        print(f'searched for {name}, and found {results[0]} as a replacement')
    elif len(results) > 1:
        multi_results[full_name] = results
    else:
        still_no_results.append(name)

searched for Femenía, and found Francisco_Femenía Far as a replacement
searched for Kanté, and found N'Golo_Kanté as a replacement
searched for Gudmundsson, and found Johann Berg_Gudmundsson as a replacement
searched for Aanholt, and found Patrick_van Aanholt as a replacement
searched for Diouf, and found Mame Biram_Diouf as a replacement
searched for Choupo-Moting, and found Eric Maxim_Choupo-Moting as a replacement
searched for Bruyne, and found Kevin_De Bruyne as a replacement
searched for Heung-Min, and found Heung-Min_Son as a replacement
searched for Chung-yong, and found Chung-yong_Lee as a replacement
searched for Hoorn, and found Mike_van der Hoorn as a replacement
searched for Dijk, and found Virgil_van Dijk as a replacement
searched for Chilwell, and found Benjamin_Chilwell as a replacement
searched for Ladapo, and found Olayinka Fredrick Oladotun_Ladapo as a replacement
searched for Sung-yueng, and found Sung-yueng_Ki as a replacement
searched for Mário, and found João Mári

In [266]:
len(still_no_results)

17

In [267]:
len(multi_results)

44

In [269]:
for k,v in multi_results.items():
    try:
        name_replace_dict[k]
        continue
        #skip this player if they've already been added to the replacement dict
    except:
        team = xg_df[(xg_df.player == k)].tail(10)['home_team'].value_counts().index[0]
        print(f'{k}, {team}')
        for idx, name in enumerate(v):
            print(f'{idx}: {name}')
        inp = ''
        while not inp.isdigit():
            inp = input('Index of Correct Match: ')
            if inp == 'exit':
                break
            elif inp == 'skip':
                continue
            elif inp == 'g':
                query = f'https://www.google.com/search?igu=1&ei=&q={k}+epl+{team}'
                display(IFrame(query, width='100%', height=300))
        name_replace_dict[k] = v[int(inp)]
        print(f'Added {k} as {v[int(inp)]} \n')

Willian, Arsenal
0: William_Saliba
1: Willian_Borges Da Silva
2: Neco_Williams
3: Rhys_Williams
4: Brandon_Williams
5: William_Smallbone
6: Willian José_Da Silva
7: Ashley_Williams
8: Jonathan_Williams
9: Joe_Williams
10: Danny_Williams


Index of Correct Match:  1


Added Willian as Willian_Borges Da Silva 

Bojan, Everton
0: Bojan_Krkic
1: Bojan_Krkic Perez


Index of Correct Match:  1


Added Bojan as Bojan_Krkic Perez 

Bruno, Brighton
0: Bruno Miguel_Borges Fernandes
1: Zak_Brunt
2: Bruno André_Cavaco Jordao
3: Chris_Brunt
4: Bruno_Martins Indi
5: Bruno_Saltor Grau
6: Bruno_Ecuele Manga


...

Index of Correct Match:  11


Added Willian_José as Willian José_Da Silva 

Sidnei_Tavares, Brighton
0: André Filipe_Tavares Gomes
1: Fabio Henrique_Tavares
2: Nikola_Tavares


Index of Correct Match:  g


Index of Correct Match:  1


Added Sidnei_Tavares as Fabio Henrique_Tavares 



In [270]:
name_replace_dict['Bruno'] = 'Bruno_Saltor Grau'
name_replace_dict['Sidnei_Tavares'] = 'Sidnei_Tavares'

In [276]:
name_replace_dict['Thiago_Silva'] = 'Thiago_Thiago'


In [279]:
xg_df.player = xg_df.player.replace(to_replace = name_replace_dict)

In [285]:
[player for player in xg_df.player.unique() if player not in df.player.unique()]

['Zanka',
 'Fernandinho',
 'Chicharito',
 'Joselu',
 'Jorginho',
 'Romain_Saiss',
 'Fabinho',
 'Nicolas_Pepe',
 'Dara_O&#039;Shea',
 'Tanguy_NDombele_Alvaro',
 'Jack_O&#039;Connell',
 'Vitinha',
 'Trézéguet',
 'Raphinha',
 'Daniel_N&#039;Lundulu',
 'Martin_Odegaard',
 'Shola_Shoretire',
 'Sidnei_Tavares']

In [302]:
name_replace_dict['Fernandinho'] = 'Fernando_Luiz Rosa'
name_replace_dict['Jorginho'] = 'Jorge Luiz_Frello Filho'
name_replace_dict['Romain_Saiss'] = 'Romain_Saïss'
name_replace_dict['Fabinho'] = 'Fabio Henrique_Tavares'
name_replace_dict['Nicolas_Pepe'] = 'Nicolas_Pépé'
name_replace_dict['Tanguy_NDombele_Alvaro'] = 'Tanguy_Ndombele'
name_replace_dict['Dara_O&#039;Shea'] = "Dara_O'Shea"
name_replace_dict['Vitinha'] = 'Vitor_Ferreira'
name_replace_dict['Trézéguet'] = 'Mahmoud Ahmed_Ibrahim Hassan'
name_replace_dict['Raphinha'] = 'Raphael_Dias Belloli'
name_replace_dict['Martin_Odegaard'] = 'Martin_Ødegaard'

In [303]:
xg_df.player = xg_df.player.replace(to_replace = name_replace_dict)

In [304]:
[player for player in xg_df.player.unique() if player not in df.player.unique()]

['Zanka',
 'Chicharito',
 'Joselu',
 'Jack_O&#039;Connell',
 'Daniel_N&#039;Lundulu',
 'Shola_Shoretire',
 'Sidnei_Tavares']

In [259]:
df['date'] = df.kickoff_time.dt.date

In [260]:
xg_df['date'] = xg_df.datetime.dt.date

In [318]:
merged = df.merge(xg_df, how = 'left', on = ['date', 'player'])

In [319]:
merged.isna().sum()

element              121268
opponent_team             0
total_points              0
was_home                  0
kickoff_time              0
team_h_score              0
team_a_score              0
gw                        0
minutes                   0
goals_scored              0
assists_x                 0
clean_sheets              0
goals_conceded            0
own_goals_x               0
penalties_saved           0
penalties_missed          0
yellow_cards              0
red_cards                 0
saves                     0
bonus                     0
bps                       0
influence                 0
creativity                0
threat                    0
ict_index                 0
transfers_balance         0
selected                  0
transfers_in              0
transfers_out             0
position_x                0
player                    0
play_proba           102378
team                      0
season                    0
finished                  0
home                

In [314]:
len(df)

105232